## Перевод из markdown в LaTeX

Подразумевается, что преобразование из Markdown идет в рамках одной подсекции (subsection):
  * в секции выделяется введение
  * разделение на user stories
  * каждая user stories разделяется на acceptance criteria

Подсекция предваритльно сохраняется в `test.md`.

После преобразования контент в формате TeX  - в соответствующей ячейке.

In [ ]:
with open("test.md") as mdfile:
    orig_md = mdfile.read()

In [ ]:
def itemize(md):
    out = ""
    s = 0
    for l in md.split("\n"):
        if re.match(r'\s+\*\s', l):
            if s == 0:
                out += "\\begin{itemize}" + "\n"
                out += l.replace("  * ", "  \item ") + "\n"
                s = 1
            else:
                out += l.replace("  * ", "  \item ") + "\n"
        else:
            if s == 1:
                out += "\end{itemize}" + "\n"
                out += l + "\n"
                s = 0
            else:
                out += l + "\n"
    if s != 0:
        out += "\end{itemize}" + "\n"
    return out

In [ ]:
t = """
Несколько добавлений пользователя проходят успешно. Для каждого видео добавляется новый `personId` добавляется.

Команда ожидает 5 видео файлов. Требования к каждому из видео файлов:
  * В первом по списку видеофайле лицо человека должно быть неподвижно (допускаются небольшие повороты)
  * Во втором по списку видеофайле должны быть зафиксированы наклоны головы влево и вправо (_roll_)
  * В третьем по списку видеофайле должны быть зафиксированы повороты головы влево и вправо (_yaw_)
  * В четвертом видеофайле должен быть зафиксирован открытый рот
  * В пятом - должно быть зафиксировано закрытие глаз
Несколько добавлений пользователя проходят успешно. Для каждого видео добавляется новый `personId` добавляется.

  * В третьем по списку видеофайле должны быть зафиксированы повороты головы влево и вправо (_yaw_)
  * В четвертом видеофайле должен быть зафиксирован открытый рот

Несколько добавлений пользователя проходят успешно. Для каждого видео добавляется новый `personId` добавляется.

  * В первом по списку видеофайле лицо человека должно быть неподвижно (допускаются небольшие повороты)
  * Во втором по списку видеофайле должны быть зафиксированы наклоны головы влево и вправо (_roll_)"""

print(itemize(t))

In [ ]:
def italic(md):
    spos = 0
    new_md = ""
    look_starting_from = spos
    while True:
        npos = md.find("_", look_starting_from)
        if npos == -1:
            new_md += md[spos:]
            #print(new_md)
            break
        elif (npos == 0 or \
             not (md[npos-1:npos].isalpha() or md[npos-1:npos].isdigit())) and \
             md[npos-1:npos] != '_' and \
             (md[npos+1:npos+2].isalpha() or md[npos+1:npos+2].isdigit()):
            stoppos = md.find("_", npos+1)
            if stoppos != -1:
                if not (md[stoppos+1:stoppos+2].isalpha() or md[stoppos+1:stoppos+2].isdigit()) and \
                   (md[stoppos-1:stoppos].isalpha() or md[stoppos-1:stoppos].isdigit()):
                    look_starting_from = stoppos + 1
                    new_md += md[spos:npos] + "\\textit{" + md[npos+1:stoppos] + "}"
                    spos = look_starting_from
                else:
                    look_starting_from = npos + 1
            else:
                look_starting_from = npos + 1
        else:
            look_starting_from = npos + 1
    return new_md

In [ ]:
a = "asdfas __1234__ asdfa\n_111_ _1234_ asdfs (_asdadf_) $ad_sa$ _adads_ _sfadf sasdfa _asdfa_ sd "
#a = " _sfadf sasdfa _asdfa_ sd "
print(italic(a))

In [ ]:
def mono(md):
    spos = 0
    new_md = ""
    look_starting_from = spos
    while True:
        npos = md.find("`", look_starting_from)
        if npos == -1:
            new_md += md[spos:]
            #print(new_md)
            break
        elif (npos == 0 or \
             not (md[npos-1:npos].isalpha() or md[npos-1:npos].isdigit())) and \
             md[npos-1:npos] != '`' and \
             (md[npos+1:npos+2].isalpha() or md[npos+1:npos+2].isdigit()):
            stoppos = md.find("`", npos+1)
            if stoppos != -1:
                if not (md[stoppos+1:stoppos+2].isalpha() or md[stoppos+1:stoppos+2].isdigit()) and \
                   (md[stoppos-1:stoppos].isalpha() or md[stoppos-1:stoppos].isdigit()):
                    look_starting_from = stoppos + 1
                    new_md += md[spos:npos] + "\\texttt{" + md[npos+1:stoppos] + "}"
                    spos = look_starting_from
                else:
                    look_starting_from = npos + 1
            else:
                look_starting_from = npos + 1
        else:
            look_starting_from = npos + 1

    return new_md

In [ ]:
a = "asdfas ```1234``` asdfa\n`111` `1234` asdfs (`asdadf`) $ad`sa$ `adads` `sfadf sasdfa `asdfa` sd "
#a = " _sfadf sasdfa _asdfa_ sd "
print(mono(a))

In [ ]:
def limit_page_width(md):
    out = ""
    lines = md.split("\n")
    for l in lines:
        if len(l) > 73:
            c = 0
            while c < len(l):
                out += l[c:c+73] + "\n"
                c += 73
        else:
            out += l + "\n"
    return(out.strip())

In [ ]:
def acceptance_io_example(md, pos):
    spos = pos + 1
    npos = md.find("```", spos)
    stoppos = md.find("\n", npos+1)
    spos = stoppos + 1
    npos = md.find("```", spos)
    out = limit_page_width(md[spos:npos].strip())
    return out, npos + 3

def acceptance_comment(md, pos):
    #print(pos)
    #print(md)
    spos = pos + 1

    #print(md[pos+1:pos+20])
    npos = md.find("_Комментарий_:\n", spos)
    #print(md[npos:pos+15])

    stoppos = md.find("\n", npos+15)
    spos = stoppos + 1
    npos = md.find("#", spos)
    #print("'" + md[spos:npos-1].strip() + "'")
    if npos == -1:
        return md[spos:].strip(), len(md)
    else:
        return md[spos:npos-1].strip(), npos - 1

def acceptance(md):
    spos = 0
    new_md = ""
    while True:
        npos = md.find("##### Критерий", spos)
        if npos == -1:
            new_md += md[spos:]
            #print(new_md)
            break
        elif npos == 0 or md[npos-1:npos] in ['\n']:
            stoppos = md.find("\n", npos+1)
            crit_id = md[stoppos-10:stoppos].strip()
            #print(crit_id)
            #print(npos, crit_id, stoppos)
            io_example, stoppos = acceptance_io_example(md, stoppos)
            #print(stoppos)
            comment, stoppos = acceptance_comment(md, stoppos)
            #print(stoppos)
            comment = mono(comment)
            comment = italic(comment)
            comment = itemize(comment)
            #print(io_example, comment)
            new_md += md[spos:npos] + \
                f'% {crit_id}\n' + \
                '\\begin{myverbbox}[\small]{\output}' + '\n' + \
                io_example + '\n'+ \
                '\end{myverbbox}' + '\n' + \
                '\\acceptanceCriteria{' + comment + '}' + '\n\n'
            spos = stoppos + 1
        else:
            spos = npos + 30

    return new_md

#print(acceptance(md))

In [ ]:
def command_example(md):
    spos = md.find("##### Использование скрипта", 0)
    if spos != -1:
        npos = md.find("```", spos + 1)
        stoppos = md.find("\n", npos+1)
        npos = md.find("```", stoppos + 1)
        out = limit_page_width(md[stoppos + 1:npos].strip())
        return out, spos, npos + 3
    return "", -1, -1

def userstory(md, pos=0):
    new_md = ""
    npos = md.find("#### US-", 0)
    while True:
        spos = npos
        if npos == -1:
            break
        elif npos == 0 or md[npos-1:npos] in ['\n']:
            us_id = md[npos + 5:npos + 11]
            lineend = md.find("\n", npos+1)
            us_name = md[npos + 12:lineend]

            stoppos = md.find("#### US-", lineend)
            if stoppos == -1:
                wc_md = md[lineend:]
            else:
                wc_md = md[lineend:stoppos-1]
            first_cr_pos = wc_md.find("##### Критерий", 0)
            if first_cr_pos != -1:
                us_intro = wc_md[:first_cr_pos]
            else:
                us_intro = wc_md
            cmd_example, sc, ec = command_example(us_intro)
            if sc != -1:
                us_cmd_comment = us_intro[ec:].strip()
                us_cmd_comment = mono(us_cmd_comment)
                us_cmd_comment = italic(us_cmd_comment)
                us_cmd_comment = itemize(us_cmd_comment)
                formatted_pre_intro = mono(us_intro[:sc])
                formatted_pre_intro = italic(formatted_pre_intro)
                formatted_pre_intro = itemize(formatted_pre_intro)
                new_us_intro = formatted_pre_intro + \
                    "\\begin{myverbbox}[\small]{\cmdLine}" + "\n" + \
                    cmd_example + "\n" + \
                    "\end{myverbbox}" + "\n" + \
                    "\scriptExample{" + "\n" + \
                    us_cmd_comment + "\n" + \
                    "}" + "\n"
            else:
                new_us_intro = mono(us_intro)
                new_us_intro = italic(new_us_intro)
            new_md += "%-----------------------------------------------------" + "\n" + \
                "%" + us_id + "\n" + \
                "\\newuserstory{" + us_name + "}" + "\n" + \
                new_us_intro + "\n"
            #print(us_id)
            if first_cr_pos != -1:
                new_md += acceptance(wc_md[first_cr_pos:])
            npos = stoppos
        else:
            npos = npos + 8

    return new_md

#print(userstory(md))

In [ ]:
def syntax(md):
    spos = md.find("**Синтаксис**", 0)
    if spos != -1:
        npos = md.find("```", spos + 1)
        stoppos = md.find("\n", npos+1)
        npos = md.find("```", stoppos + 1)
        return md[stoppos + 1:npos].strip(), spos, npos + 3
    return "", -1, -1

def subsection(md):
    new_md = ""
    npos = md.find("### ", 0)
    if npos == 0 or md[npos-1:npos] in ['\n']:
        lineend = md.find("\n", npos+1)
        subsection_name = md[npos+4:lineend]
    else:
        raise Exception("Cannot find a subsection")
    intro = "\subsection*{" + subsection_name + "}" + "\n"
    first_us_pos = md.find("#### US-", lineend)
    if first_us_pos == -1:
        raise Exception("No user story in the subsection")
    wc_md = md[lineend:first_us_pos]
    cmd, sc, ec = syntax(wc_md)
    if sc != -1:
        intro += wc_md[:sc] + \
            "\\begin{myverbbox}{\scriptFile}" + "\n" + \
            cmd + "\n" + \
            "\end{myverbbox}" + "\n" + \
            "\scriptTitle" + "\n" + \
            wc_md[ec:]
    else:
        intro += wc_md
        
    section = intro + userstory(md[first_us_pos:])
    
    return(section)
        
#print(subsection(md))

In [ ]:
import re

def insert_backslash(match):
    c = match.group()
    return(c[0]+"\\"+c[1])

def replace_percentage(md):
    p = re.compile(r'[^\\]%', re.VERBOSE)
    return p.sub(insert_backslash, md)

In [ ]:
def find_json_code(md):
    npos = md.find("```json", 0)
    if npos != -1:
        lineend = md.find("\n", npos+1)
        spos = md.find("```", npos+3)
        s = md[lineend:spos].strip()
        lineend = md.find("\n", spos+1)
        return (s, npos, lineend)
    else:
        return "", -1, -1

def replace_json_code(md):
    js, sc, ec = find_json_code(md)
    while sc != -1:
        js = js.replace("}", "\string}")
        js = js.replace("{", "\string{")
        md = md[:sc] + "\\texttt{" + js + "}\n" + md[ec:]
        js, sc, ec = find_json_code(md)
    return md

In [ ]:
def find_solidity_code(md):
    npos = md.find("```solidity", 0)
    if npos != -1:
        lineend = md.find("\n", npos+1)
        spos = md.find("```", npos+3)
        s = md[lineend:spos].strip()
        lineend = md.find("\n", spos+1)
        return (s, npos, lineend)
    else:
        return "", -1, -1

def replace_solidity_code(md):
    sol, sc, ec = find_solidity_code(md)
    while sc != -1:
        md = md[:sc] + "\\texttt{" + sol + "}\n" + md[ec:]
        sol, sc, ec = find_solidity_code(md)
    return md

In [ ]:
md = replace_percentage(orig_md)
md = replace_json_code(md)
md = replace_solidity_code(md)

In [ ]:
#print(md)
print(subsection(md))